# Flink Demo

## Installation

In [2]:
try:
    import google.colab
    GOOGLE_COLAB = True
except ImportError:
    GOOGLE_COLAB = False
    
GOOGLE_COLAB

False

### Install Java and Flink

In [6]:
if GOOGLE_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
else:
    !which java

/usr/local/buildtools/java/jdk/bin/java


In [57]:
!wget -q https://www-eu.apache.org/dist/flink/flink-1.8.0/flink-1.8.0-bin-scala_2.12.tgz -O flink-1.8.0-bin-scala_2.12.tgz
!tar xf flink-1.8.0-bin-scala_2.12.tgz
!rm flink-1.8.0-bin-scala_2.12.tgz

### Start a Flink cluster

In [59]:
%%bash
./flink-1.8.0/bin/start-cluster.sh

Starting cluster.
Starting standalonesession daemon on host strokach.svl.corp.google.com.
Starting taskexecutor daemon on host strokach.svl.corp.google.com.


### (Optional) Create a tunnel to the Flink dashboard

In [60]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [61]:
if GOOGLE_COLAB:
    import json
    import os
    import time
    import subprocess
    import sys
    from urllib import request

    env = os.environ.copy()
    if os.getcwd() not in env["PATH"]:
        env["PATH"] = "{}:{}".format(os.getcwd(), env["PATH"])
    ngrok = subprocess.Popen(["ngrok", "http", "8081"], env=env)

    time.sleep(1)
    link = "http://localhost:4040/api/tunnels"
    ngrok_dashboard = json.loads(request.urlopen(link).read())['tunnels']
    for tunnel in json.loads(request.urlopen(link).read())['tunnels']:
        print(tunnel["name"], tunnel["public_url"])
else:
    print("http://localhost:8081")

### Install Apache Beam

In [62]:
!git clone https://github.com/apache/beam.git

Cloning into 'beam'...
remote: Enumerating objects: 403640, done.
remote: Total 403640 (delta 0), reused 0 (delta 0), pack-reused 403640
Receiving objects: 100% (403640/403640), 126.25 MiB | 21.41 MiB/s, done.
Resolving deltas: 100% (187592/187592), done.
Updating files: 100% (5837/5837), done.


In [1]:
!pip install -q virtualenv

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


In [6]:
!python --version

Python 2.7.15


In [7]:
!pushd beam && chmod +x ./gradlew ./gradlew_orig && ./gradlew :sdks:python:container:docker > sdks-python-container-docker-build.log

~/DriveFS/Colab Notebooks/beam-notebooks/notebooks/beam ~/DriveFS/Colab Notebooks/beam-notebooks/notebooks
Traceback (most recent call last):
  File "/usr/local/google/home/strokach/miniconda3/lib/python3.7/site-packages/virtualenv.py", line 2884, in <module>
    main()
  File "/usr/local/google/home/strokach/miniconda3/lib/python3.7/site-packages/virtualenv.py", line 712, in main
    symlink=options.symlink)
  File "/usr/local/google/home/strokach/miniconda3/lib/python3.7/site-packages/virtualenv.py", line 927, in create_environment
    site_packages=site_packages, clear=clear, symlink=symlink))
  File "/usr/local/google/home/strokach/miniconda3/lib/python3.7/site-packages/virtualenv.py", line 1389, in install_python
    os.symlink(py_executable_base, full_pth)
OSError: [Errno 38] Function not implemented

FAILURE: Build completed with 3 failures.

1: Task failed with an exception.
-----------
* What went wrong:
Execution failed for task ':sdks:python:container:goPrepare'.
> Create sy

In [71]:
!pwd

/usr/local/google/home/strokach/DriveFS/Colab Notebooks/beam-notebooks/notebooks
